# A project in bioinformatics

## Part 1

In [1]:
from collections import Counter
from collections import defaultdict
import timeit

import os
WORKINNG_DIR = os.getcwd()

WORDS_PLASMID_TXT = os.path.join(WORKINNG_DIR, r"cog_words_plasmid.txt")
WORDS_BAC_TXT = os.path.join(WORKINNG_DIR, r"cog_words_bac.txt")


### 1.1. Find cog problem: Finding the genomes in which the cog appears
The function receives a genome and check if the cog appears in it.

- Input: genome, HashMap of wordes, cog.

In [2]:
# take one row from DB and check if cog appears in it
def find_cog(data, cog_map, cog, l):
    for row in data:
        for i in range(5, len(row)):
            if cog == row[i]:
                find_words(cog_map, row, i, l)  # find all neighbors of cog
                break
                

### 1.2. Find Wordes: Finding all of cog's neighbors
The function divides the genome into l-length segments and checks if it appears in each segment.
If so, it puts the word in the Hashmap

- Input: HashMap of words, genome, index of cog.

In [3]:
# find all words (cog+neighbors) in a specific row
def find_words(cog_map, row, i, l):
    for l_param in range(2, l + 1):  # number of neighbours 2-10
        if l_param <= len(row) - 5:  # 5 is the index of the first cog in the row
            for j in range(5, len(row) - l_param + 1):
                word = tuple(row[j: j + l_param])  # tuple is hashable (the length of word is l_param)
                if row[i] in word:  # if the cog is in the word
                    if word in cog_map:  # if the word is already in the hashmap
                        value_of_word = cog_map[word]
                        value_of_word[0] += 1
                        organism = row[3]  # row[3] is the organism's name
                        value_of_word[1].add(organism)  # add the organism's name to the set
                        cog_map[word] = value_of_word  # set to the map the new value of key 'word'
                    else:
                        cog_map[word] = [1, {row[3]}]  # add a new key 'word' to the map


### 1.3. bigger_than_q: Find Wordes that are  conserved in ≥ 𝑞 of the genomes
The function receives a map of all the words and checks if the size of its group of organisms is greater than q.
If so, it puts it in a counter with the size

- Input: HashMap of words, q, counter that save the wordes and the number genomes in which it appears.

In [4]:
# filter the map and only consider words that appear in at least q different organisms
def bigger_than_q(cog_map, q, counter):
    for key in cog_map:
        value = cog_map[key]
        if len(value[1]) >= q:  # check if the size of the organisms set is at least q
            counter[key] = len(value[1])


### 1.4. sort_output: Sort the words in the counter by length and q
The function receives a counter, sorts it into sub-list according to the lengths of the word and sort each list by decreasing number of genomes in which the word was found 

- Input: counter
- output: sort list by length and q 

In [5]:
def sort_output(counter, output):
    # Inserting the words into the output by the number of their appearances in the genome, in descending order
    for k, v in counter.most_common():
        output.append(k)

    group = defaultdict(list)
    # sort the output by the length
    for c in output:
        group[len(c)].append(c)
    return group


## Part 2

### 2.1. k_instances: finding all the wordes with k-instances
The function checks for each word result from prat 1 if it appears in this result with k-insrtion. If so, it puts the word in the  new Counter 

- Input: sorted_groups( the result from part 1), sort list of words, k, counter2

In [6]:
def k_instances(sorted_groups, output, k, counter2):
    for word in output:  
        len_word = len(word)
        sum_k = 1
        check_key = len_word + sum_k
        while sum_k <= k and check_key in sorted_groups.keys():  # check if the next group(by key) is in sorted_group
            group = sorted_groups.get(check_key)
            find_word = is_find_word_k(word, group)  # number of times "word" appear in each group
            counter2[word] = counter2[word] + find_word  # insert to new Counter
            sum_k += 1
            check_key += 1

### 2.2. is_find_word_k: finding the word with k-instances in the group 
The function receives a word and a group with up to k-instances and goes over each word in the group. If it finds a suitable word it increases the amount of total_sum by 1

- Input: word and group of words with tha same length
- Output: the number of appreance the word in the group


In [7]:
def is_find_word_k(word, group):
    total_sum = 0
    for arr in group:
        if word[0] == arr[0] and word[len(word) - 1] == arr[len(arr) - 1]:  # to allow insertions only at the middle
            j = 1
            for i in range(1, len(arr)):
                if word[j] == arr[i]:
                    j += 1
                if j == len(word):  # means we find all the cogs of word in arr
                    total_sum += 1
                    break
    return total_sum

### 2.3 update_counter2: update the values in counter
The function receives 2 counters and updates the key values of counter2 to be the sum of its values in counters1 and counter2

- Input: sorted_groups (the result from part 1), sort list of words, k, counter2


In [8]:
def update_counter2(counter1, counter2): 
    for word in counter2:
        counter2[word] += counter1[word]


### 2.4. bigger_than_q2: Find Wordes that are  conserved in ≥ 𝑞 of the genomes
The function receives a counter and checks if the size of its group of organisms is greater than q.
If so, it puts it in a new counter with the size

- Input: q and counter 
- Output: counter that contains all the wordes which appear at least q.

In [9]:
def bigger_than_q2(q, counter):
    new_counter = Counter()
    for word in counter:
        count = counter[word]
        if counter[word] >= q:  # check if the size of the organisms set is at least q
            new_counter[word] = count
    return new_counter

### main

In [10]:
# open, read and split both of the files. return the words that contain the cog and appear in at least q organisms
def main(cog, q, l, k):
    # start_time = timeit.default_timer()
    # first file - plasmid
    with open(WORDS_PLASMID_TXT, "rb") as f:
        data1 = f.readlines()  # read content as lines

    # split each line by tab (\t) and remove the new line at the last cell
    split1 = [x.split(b"\t")[:-1] for x in data1]

    # split the first cell by hashtag (#) and combine with the rest of the list
    data1 = [list([*x[0].split(b"#"), *x[1:]]) for x in split1]

    counter1 = Counter()  # create a Counter with key = word, value = the size of the group that the organism appears in it (q)
    cog_map = {}  # create a hashmap with the word (a tuple) as a key and array of [number_of_appears, set_of_organism] as the value
    find_cog(data1, cog_map, cog, l)  # a map of words that contain the cog from the first DB

    # open second file - bacteria
    with open(WORDS_BAC_TXT, "rb") as f:
        data2 = f.readlines()  # read content as lines

    # split each line by tab (\t) and remove the new line at the last cell
    split2 = [x.split(b"\t")[:-1] for x in data2]

    # split the first cell by hashtag (#) and combine with the rest of the list
    data2 = [list([*x[0].split(b"#"), *x[1:]]) for x in split2]

    find_cog(data2, cog_map, cog, l)  # a map of words that contain the cog from both DB
    bigger_than_q(cog_map, 1, counter1)  # update the counter with the words that the value bigger than q
    output = []
    sorted_groups = sort_output(counter1, output)  # a  sort list of words that appear at least one
    
    if k == 0:  # no insertions
        new_counter = bigger_than_q2(q, counter1)
        output = []
        sorted_groups_k = sort_output(new_counter, output)
        print(sorted_groups_k)
    else:
        counter2 = Counter()
        k_instances(sorted_groups, output, k, counter2)
        update_counter2(counter1, counter2)
        new_counter = bigger_than_q2(q, counter2)
        output = []  # to clear the output
        sorted_groups_k = sort_output(new_counter, output)
        print(sorted_groups_k)


In [11]:
 main(b"0586", 20,5 ,2)

defaultdict(<class 'list'>, {2: [(b'X', b'0586'), (b'0586', b'X'), (b'0101', b'0586'), (b'0586', b'0777'), (b'0626', b'0586'), (b'2186', b'0586'), (b'2207', b'0586'), (b'0586', b'0586'), (b'0586', b'0260'), (b'3963', b'0586'), (b'0586', b'0322'), (b'0503', b'0586'), (b'0586', b'1322'), (b'0586', b'0861'), (b'3339', b'0586'), (b'1051', b'0586'), (b'0586', b'2244'), (b'0499', b'0586')], 3: [(b'0136', b'0101', b'0586'), (b'0101', b'0586', b'0777'), (b'0586', b'0777', b'0285'), (b'0586', b'X', b'X'), (b'X', b'X', b'0586'), (b'2271', b'2186', b'0586'), (b'X', b'0586', b'X'), (b'2186', b'0586', b'X'), (b'0586', b'0260', b'0012'), (b'2186', b'0586', b'0322'), (b'0503', b'0586', b'0260'), (b'X', b'0503', b'0586'), (b'0586', b'1322', b'2226'), (b'5337', b'1051', b'0586'), (b'1051', b'0586', b'2244'), (b'0586', b'2244', b'0598')], 4: [(b'0136', b'0101', b'0586', b'0777'), (b'0101', b'0586', b'0777', b'0285'), (b'0111', b'0136', b'0101', b'0586'), (b'0586', b'0777', b'0285', b'3147'), (b'0586', b